<a href="https://colab.research.google.com/github/YunhoCha/2024-AI-/blob/main/%EC%8B%A0%EB%AC%B8%EC%95%84%EC%B9%B4%EC%9D%B4%EB%B8%8C_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install -y wget unzip
!wget -O /content/google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i /content/google-chrome-stable_current_amd64.deb
!apt --fix-broken install -y
!google-chrome --version
!pip install selenium
!pip install pandas

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,171 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,553 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,447 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,133 kB]
Get:14 ht

In [ ]:
import requests
import zipfile
import os

# ChromeDriver 다운로드 URL 구성
chromedriver_url = "https://storage.googleapis.com/chrome-for-testing-public/127.0.6533.119/linux64/chromedriver-linux64.zip"

# ChromeDriver 다운로드
chrome_driver_zip_path = '/content/chromedriver.zip'
with requests.get(chromedriver_url, stream=True) as r:
    with open(chrome_driver_zip_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

# ChromeDriver 압축 해제
with zipfile.ZipFile(chrome_driver_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/chromedriver')

# ChromeDriver 경로 설정 및 권한 부여
chrome_driver_path = '/content/chromedriver/chromedriver-linux64/chromedriver'
os.chmod(chrome_driver_path, 0o755)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from tqdm import tqdm

# 사용자 입력
start_id_str = input("Enter the start content ID (e.g., 00093176030): ")
end_id_str = input("Enter the end content ID (e.g., 00093176035): ")

# 크롬 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 서비스 설정
service = Service(executable_path=chrome_driver_path)

# 데이터 저장 리스트
data = []

# 브라우저 시작
driver = webdriver.Chrome(service=service, options=chrome_options)

# 문자열을 정수로 변환하여 범위 생성
start_id = int(start_id_str)
end_id = int(end_id_str)

# 문자열의 길이를 유지한 채로 범위 생성
try:
    for content_id in tqdm(range(start_id, end_id + 1), desc="Progress"):
        content_id_str = str(content_id).zfill(len(start_id_str))  # 원래 문자열 길이를 유지하며 0으로 채움
        url = f"https://www.nl.go.kr/newspaper/detail.do?content_id=CNTS-{content_id_str}"
        driver.get(url)

        try:
            # 페이지가 로드될 시간을 대기
            time.sleep(5)  # 대기 시간을 5초로 증가

            # 데이터 추출
            newspaper_name = driver.find_element(By.XPATH, "//td[text()='신문명']/following-sibling::td").text
            publisher = driver.find_element(By.XPATH, "//td[text()='발행처']/following-sibling::td").text
            issued_date = driver.find_element(By.XPATH, "//td[text()='발행일']/following-sibling::td").text
            news_position = driver.find_element(By.XPATH, "//td[text()='기사위치']/following-sibling::td").text
            news_type = driver.find_element(By.XPATH, "//td[text()='기사유형']/following-sibling::td").text
            news_topic = driver.find_element(By.XPATH, "//td[text()='기사주제']/following-sibling::td").text
            description = driver.find_element(By.XPATH, "//td[text()='주기사항']/following-sibling::td").text
            abstract = driver.find_element(By.XPATH, "//td[text()='본문정보']/following-sibling::td").text

            # 데이터 저장
            data.append({
                "Content ID": content_id_str,
                "Newspaper Name": newspaper_name,
                "Publisher": publisher,
                "Issued Date": issued_date,
                "News Position": news_position,
                "News Type": news_type,
                "News Topic": news_topic,
                "Description": description,
                "Content": abstract
            })

        except Exception as e:
            print(f"Failed to retrieve data for content ID: CNTS-{content_id_str}")
            print(e)
            continue  # 예외 발생 시 다음 ID로 넘어감

finally:
    # 브라우저 종료
    driver.quit()

    # DataFrame으로 변환
    df = pd.DataFrame(data)

    # DataFrame을 엑셀 파일로 저장
    output_path = '/content/newspaper_data.xlsx'
    df.to_excel(output_path, index=False)
    print(f"Data saved to {output_path}")

    # 마지막으로 처리된 ID 출력
    if data:
        last_processed_id = data[-1]["Content ID"]
        print(f"Last processed content ID: CNTS-{last_processed_id}")



Enter the start content ID (e.g., 00093176030): 00093176030
Enter the end content ID (e.g., 00093176035): 00093176035


Progress: 100%|██████████| 6/6 [00:56<00:00,  9.38s/it]


Data saved to /content/newspaper_data.xlsx
Last processed content ID: CNTS-00093176035


In [ ]:
 # DataFrame 출력
df

,Content ID,Newspaper Name,Publisher,Issued Date,News Position,News Type,News Topic,Description,Content
0,00093176030,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,3면 3단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",大邱愛國婦人會에셔 孤兒院設施은 已往揭載얏거니와 大抵我한婦女의 財政權能 업 程度...
1,00093176031,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,3면 4단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",請垂詳覽本社에셔 美國시아틀大博覽會出品事件으로 廣告에 對야 僉彦의 多大 贊成을 ...
2,00093176032,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 1단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",京城本町郵便局前 篠崎シノサキ器械店平壤南門通二丁目 篠崎シノサキ支店
3,00093176033,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 1단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",本所에셔 織造 疋木으로 學生帽子를 精撰製造와 大發賣하오니 僉同胞 壹次試驗시오...
4,00093176034,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 1단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",器械好品 아니라 廉價로 發賣옵고 不完全 것이 有면 換付기 近郡僉員은 購...
5,00093176035,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 2단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",鑿井而飮고 耕田而食랴 얏더니 堯舜世界 머럿셔라 無去處之茫茫이로다出將入相야 輔...
